In [ ]:
# ref1 : https://www.kaggle.com/chayan8/sentiment-analysis-using-bert-pytorch/notebook
# ref2 : https://www.kaggle.com/anik424/sentiment-analysis-multiclassification-using-bert/notebook

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 1.4 MB/s 
     |████████████████████████████████| 6.8 MB 20.3 MB/s 
     |████████████████████████████████| 981 kB 56.6 MB/s 
     |████████████████████████████████| 263 kB 58.9 MB/s 
     |████████████████████████████████| 2.8 MB 55.5 MB/s 
     |████████████████████████████████| 1.2 MB 48.3 MB/s 
     |████████████████████████████████| 468 kB 60.4 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 895 kB 48.2 MB/s 
     |████████████████████████████████| 3.3 MB 49.1 MB/s 
     |████████████████████████████████| 61 kB 481 kB/s 
  Created wheel for ktrain: filename=ktrain-0.28.3-py3-none-any.whl size=25292659 sha256=8ddf5e84e65671add53d2c800fb33ce2693b6a0a841feaefa87204c44146d7c2
  Stored in directory: /root/.cache/pip/wheels/6a/7e/c3/f46cdfc2b81c54424923b1405d7e670c35cacc11ada9a47b1c
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=425018df47abc887

In [ ]:
import pandas as pd
import numpy as np
import ktrain
import tensorflow as tf
from ktrain import text
from sklearn.model_selection import train_test_split
import collections

In [ ]:
tf.version.VERSION

## Preparing our train and test dataset

In [ ]:
data = pd.read_feather('/content/drive/MyDrive/Capstone/data/sentiment140_text_sentiment_preprocessed.ftr')

In [ ]:
data.head()

,Text,Top5%,Emoji_1_ind,Emoji_2_ind,Emoji_3_ind,Emoji_4_ind,Emoji_5_ind,Emoji_1,Emoji_2,Emoji_3,Emoji_4,Emoji_5,Pct_1,Pct_2,Pct_3,Pct_4,Pct_5,Emoji_ind,Emoji
0,hey USER looked lyric end really like listen h...,0.191943246871233,0,41,28,0,49,smile,eyes,see_no_evil,smile,speak_no_evil,0.043602,0.039773,0.037269,0.035978,0.035320,0,smile
1,USER hi demi like real name,0.19123656302690506,0,58,50,9,26,smile,facepunch,wink,smirk,information_desk_person,0.041023,0.040122,0.037898,0.036937,0.035258,0,smile
2,USER tear let cereal lmao,0.31485196575522423,0,2,2,51,2,smile,sob,sob,skull,sob,0.093497,0.062283,0.061547,0.049416,0.048108,2,sob
3,honnestly know ask friend go europe,0.18845690600574017,0,51,9,0,0,smile,skull,smirk,smile,smile,0.055959,0.039506,0.034187,0.029511,0.029294,0,smile
4,USER would said bel 6gag friend acquaintance r...,0.23708048090338707,0,39,1,0,25,smile,mask,angry,smile,neutral_face,0.075699,0.045222,0.041067,0.038921,0.036172,0,smile


In [ ]:
data.shape

(1461183, 19)

In [ ]:
# # index int형으로 변환
# data = data.astype({'Emoji_1_ind' : 'int', 'Emoji_2_ind' : 'int','Emoji_3_ind' : 'int','Emoji_4_ind' : 'int','Emoji_5_ind' : 'int'})

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461183 entries, 0 to 1461182
Data columns (total 19 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Text         1461183 non-null  object 
 1   Top5%        1461183 non-null  object 
 2   Emoji_1_ind  1461183 non-null  int64  
 3   Emoji_2_ind  1461183 non-null  int64  
 4   Emoji_3_ind  1461183 non-null  int64  
 5   Emoji_4_ind  1461183 non-null  int64  
 6   Emoji_5_ind  1461183 non-null  int64  
 7   Emoji_1      1461183 non-null  object 
 8   Emoji_2      1461183 non-null  object 
 9   Emoji_3      1461183 non-null  object 
 10  Emoji_4      1461183 non-null  object 
 11  Emoji_5      1461183 non-null  object 
 12  Pct_1        1461183 non-null  float64
 13  Pct_2        1461183 non-null  float64
 14  Pct_3        1461183 non-null  float64
 15  Pct_4        1461183 non-null  float64
 16  Pct_5        1461183 non-null  float64
 17  Emoji_ind    1461183 non-null  int64  
 18  Em

In [ ]:
# Text 결측치 처리
# data = data.dropna(axis=0)

In [ ]:
# Train : test = 99 : 1
X_train, X_test, y_train, y_test = train_test_split(data.index.values,
                                                  data.Emoji.values, 
                                                  test_size=0.01, 
                                                  random_state=42,
                                                  stratify= data.Emoji.values)

In [ ]:
# data_type == train or test
data['data_type'] = ['not_set']*data.shape[0]

In [ ]:
data.head()

,Text,Top5%,Emoji_1_ind,Emoji_2_ind,Emoji_3_ind,Emoji_4_ind,Emoji_5_ind,Emoji_1,Emoji_2,Emoji_3,Emoji_4,Emoji_5,Pct_1,Pct_2,Pct_3,Pct_4,Pct_5,Emoji_ind,Emoji,data_type
0,hey USER looked lyric end really like listen h...,0.191943246871233,0,41,28,0,49,smile,eyes,see_no_evil,smile,speak_no_evil,0.043602,0.039773,0.037269,0.035978,0.035320,0,smile,not_set
1,USER hi demi like real name,0.19123656302690506,0,58,50,9,26,smile,facepunch,wink,smirk,information_desk_person,0.041023,0.040122,0.037898,0.036937,0.035258,0,smile,not_set
2,USER tear let cereal lmao,0.31485196575522423,0,2,2,51,2,smile,sob,sob,skull,sob,0.093497,0.062283,0.061547,0.049416,0.048108,2,sob,not_set
3,honnestly know ask friend go europe,0.18845690600574017,0,51,9,0,0,smile,skull,smirk,smile,smile,0.055959,0.039506,0.034187,0.029511,0.029294,0,smile,not_set
4,USER would said bel 6gag friend acquaintance r...,0.23708048090338707,0,39,1,0,25,smile,mask,angry,smile,neutral_face,0.075699,0.045222,0.041067,0.038921,0.036172,0,smile,not_set


In [ ]:
data.loc[X_train, 'data_type'] = 'train'
data.loc[X_test, 'data_type'] = 'test'

In [ ]:
# 감정 별 train, test 데이터 개수

data.groupby(['Emoji', 'data_type']).count()

Text   Top5%  Emoji_1_ind  ...   Pct_4   Pct_5  Emoji_ind
Emoji data_type                               ...                           
angry test         1594    1594         1594  ...    1594    1594       1594
      train      157819  157819       157819  ...  157819  157819     157819
smile test         9033    9033         9033  ...    9033    9033       9033
      train      894281  894281       894281  ...  894281  894281     894281
sob   test         3985    3985         3985  ...    3985    3985       3985
      train      394471  394471       394471  ...  394471  394471     394471

[6 rows x 18 columns]

In [ ]:
# train, test split

df_train = data[data['data_type'] == 'train']
df_test = data[data['data_type'] == 'test']

In [ ]:
df_train.shape

(1446571, 20)

In [ ]:
df_test.shape

(14612, 20)

In [ ]:
X_train = df_train['Text'].tolist()
X_test = df_test['Text'].tolist()
y_train = df_train['Emoji_ind'].tolist()
y_test = df_test['Emoji_ind'].tolist()

In [ ]:
# Checking size of our train and test datasets
print(len(X_train),len(X_test),len(y_train),len(y_test))

1446571 14612 1446571 14612


In [ ]:
# Encoding our sentiment categories into numeric values
encoding = dict(zip(data.Emoji, data.Emoji_ind))
sorted_by_value = sorted(encoding.items(), key=lambda x: x[1])

encoding = collections.OrderedDict(sorted_by_value)
encoding

OrderedDict([('smile', 0), ('angry', 1), ('sob', 2)])

In [ ]:
# Our dataset has below categories/factors
factors = list(encoding.keys())
factors

['smile', 'angry', 'sob']

## Building Model using Transformer

In [ ]:
model_arch = 'albert-base-v2'
# model_arch ='bert-base-uncased'
MAXLEN = 64
trans = text.Transformer(model_arch, maxlen=MAXLEN, class_names= factors)

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [ ]:
train_data = trans.preprocess_train(X_train, y_train)
test_data = trans.preprocess_test(X_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 8
	95percentile : 15
	99percentile : 17


Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 8
	95percentile : 15
	99percentile : 17


In [ ]:
# 처음부터 시작할 때
# model = trans.get_classifier()

Downloading:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

In [ ]:
model = ktrain.load_predictor('/content/drive/MyDrive/Capstone/ktrain_predictor/').model

In [ ]:
learner = ktrain.get_learner(model, train_data=train_data, val_data=test_data, batch_size=64)

In [ ]:
learner.lr_find(show_plot=True, max_epochs=5)

In [ ]:
learner.fit_onecycle(lr = 4e-6, epochs = 5, checkpoint_folder='/content/drive/MyDrive/Capstone/ktrain_learner/')

In [ ]:
# # load weights into model
learner.model.load_weights('/content/drive/MyDrive/Capstone/ktrain_learner/weights-load.hdf5')

In [ ]:
# # save model
learner.save_model('/content/drive/MyDrive/Capstone/ktrain_model')

In [ ]:
# # load model
# learner.load_model('/content/drive/MyDrive/Capstone/ktrain_model', trans)

### Confusion Matrix

In [ ]:
learner.validate(val_data=test_data, class_names=factors)

              precision    recall  f1-score   support

       smile       0.79      0.86      0.83      9033
       angry       0.49      0.31      0.38      1594
         sob       0.63      0.61      0.62      3985

    accuracy                           0.73     14612
   macro avg       0.64      0.59      0.61     14612
weighted avg       0.72      0.73      0.72     14612



array([[7760,  271, 1002],
       [ 699,  496,  399],
       [1313,  247, 2425]])

In [ ]:
learner.validate(val_data=test_data, class_names=factors)

### loss top 10 data 

In [ ]:
learner.view_top_losses(n=10, preproc=trans, val_data=test_data)

----------
id:12999 | loss:3.97 | true:angry | pred:smile)

----------
id:5726 | loss:3.95 | true:angry | pred:smile)

----------
id:5706 | loss:3.86 | true:angry | pred:smile)

----------
id:436 | loss:3.85 | true:angry | pred:smile)

----------
id:5442 | loss:3.85 | true:angry | pred:smile)

----------
id:464 | loss:3.84 | true:angry | pred:smile)

----------
id:5466 | loss:3.83 | true:angry | pred:smile)

----------
id:4933 | loss:3.82 | true:angry | pred:smile)

----------
id:10611 | loss:3.79 | true:angry | pred:smile)

----------
id:12732 | loss:3.78 | true:angry | pred:smile)



In [ ]:
top_loss_10 = [12999, 5726, 5706, 436, 5442, 464, 5466, 4933, 10611, 12732]
for ind in top_loss_10:
  print(X_test[ind])

eternally grateful god given friend family keep sane boost ego 
USER hey checkin 
USER loving new avatar 
USER awesome awesome awesome want try every flavor amp every kind send hehe laters enjoy ehugs yay 
learning social medium expert 
USER URL congrats jade hope great time lot fun 
almost forgot wholehearted elephant arrived today love thank mati thank brene 
USER hope day fun tastic 
USER new link look nice clickable 
USER super news scott congrats 


In [ ]:
data[data['Text'].isin([X_test[5726]])]

,Text,Top5%,Emoji_1_ind,Emoji_2_ind,Emoji_3_ind,Emoji_4_ind,Emoji_5_ind,Emoji_1,Emoji_2,Emoji_3,Emoji_4,Emoji_5,Pct_1,Pct_2,Pct_3,Pct_4,Pct_5,Emoji_ind,Emoji,data_type
566659,USER hey checkin,0.45797182619571686,1,1,58,1,42,angry,angry,facepunch,angry,gun,0.11255,0.096771,0.0961,0.079157,0.073393,1,angry,test


### Predictor 저장

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [ ]:
inp = 'I am very disappointed with this kind of front camera. Need refund.'

In [ ]:
predictor.predict(inp)

'sob'

In [ ]:
predictor.save('/content/drive/MyDrive/Capstone/ktrain_predictor/')

### predictor 불러오기

In [ ]:
load_predictor = ktrain.load_predictor('/content/drive/MyDrive/Capstone/ktrain_predictor/')

In [ ]:
load_predictor.model

In [ ]:
load_predictor.preproc

In [ ]:
inp1 = 'I am very disappointed with this kind of front camera. Need refund.'
inp2 = "I really hate you"
inp3 = 'The world looks beautiful'

In [ ]:
load_predictor.predict(inp1)

'sob'

In [ ]:
load_predictor.predict(inp2)

'angry'

In [ ]:
load_predictor.predict(inp3)

'smile'